# Gold & Silver Data Scraper for Investing.com


## Goal: Scrape latest daily price from Investing.com and save the data in a csv format


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
from urllib.request import Request, urlopen


### The urls we will be scraping the data from:

In [2]:
url_gold = 'https://www.investing.com/commodities/gold-historical-data'
url_silver = 'https://www.investing.com/commodities/silver-historical-data'


### Gives our program permission to access the web page:

In [3]:
req_gold = Request(url_gold, headers={'User-Agent': 'Mozilla/5.0'})
req_silver = Request(url_silver, headers={'User-Agent': 'Mozilla/5.0'})

### Access the webpage and grab our soup data

In [4]:
webpage_gold = urlopen(req_gold).read()
webpage_silver = urlopen(req_silver).read()

soup_gold = BeautifulSoup(webpage_gold,'html.parser')
soup_silver = BeautifulSoup(webpage_silver,'html.parser')


### Creating the web scraper which takes in the soup and grabs relevent data

In [5]:
def scraper(soup):
    count = 0
    date = []
    price = []
    open = []
    high = []
    low = []
    volume = []
    change = []
    
    for table in soup.find_all('table',{'class':'genTbl closedTbl historicalTbl'}):
        for tr in table.find_all('tr'):
            for td in tr.find_all('td'):
                
                if count == 0:
                    formatted_date = datetime.datetime.strptime(td.text, '%b %d, %Y').strftime('%Y-%m-%d')
                    date.append(formatted_date)
                elif count == 1:
                    price.append(td.text)
                elif count == 2:
                    open.append(td.text)
                elif count == 3:
                    high.append(td.text)
                elif count == 4:
                    low.append(td.text)
                elif count == 5:
                    volume.append(td.text)
                elif count == 6:
                    change.append(td.text)
                    
                count += 1
                
                if count == 7:
                    count = 0
    df = list(zip(date,price,open,high,low,volume, change))
    df = pd.DataFrame(df, columns = ['Date','Price','Open','High','Low','Volume','Change'])
    return df


### Create the dataframe for each commodity and assign the columns

In [6]:
df_gold = scraper(soup_gold)
df_silver = scraper(soup_silver)

### Save the data to a csv file

In [7]:
df_gold.to_csv('Gold.csv')
df_silver.to_csv('Silver.csv')